In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from tensorflow import keras

In [2]:
root_logdir = os.path.join(os.curdir, "my_tb_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [3]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

In [4]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [5]:
from functools import partial

In [6]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 8
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [7]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [9]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [10]:
history = model.fit(train_set, steps_per_epoch=int(0.75 * dataset_size / batch_size), validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size), epochs=5)

Epoch 1/5
344/344 [==============================] - 31s 75ms/step - loss: 3.4713 - accuracy: 0.8129 - val_loss: 2.5569 - val_accuracy: 0.8309
Epoch 2/5
344/344 [==============================] - 25s 74ms/step - loss: 1.0149 - accuracy: 0.9092 - val_loss: 2.0090 - val_accuracy: 0.8401
Epoch 3/5
344/344 [==============================] - 26s 74ms/step - loss: 0.6761 - accuracy: 0.9190 - val_loss: 1.6326 - val_accuracy: 0.8566
Epoch 4/5
344/344 [==============================] - 26s 75ms/step - loss: 0.5602 - accuracy: 0.9288 - val_loss: 1.4881 - val_accuracy: 0.8566
Epoch 5/5
344/344 [==============================] - 26s 75ms/step - loss: 0.3476 - accuracy: 0.9415 - val_loss: 1.4506 - val_accuracy: 0.8658


In [11]:
tf.config.experimental.get_memory_info('GPU:0')

{'current': 94383616, 'peak': 1883904768}

In [12]:
for layer in base_model.layers:
    layer.trainable=True
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, steps_per_epoch=int(0.75 * dataset_size / batch_size), validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size), epochs=5, callbacks=[tensorboard_cb])

Epoch 1/5
344/344 [==============================] - 107s 298ms/step - loss: 1.1120 - accuracy: 0.5763 - val_loss: 0.8526 - val_accuracy: 0.6415
Epoch 2/5
344/344 [==============================] - 103s 299ms/step - loss: 0.5795 - accuracy: 0.7889 - val_loss: 0.4080 - val_accuracy: 0.8695
Epoch 3/5
344/344 [==============================] - 104s 301ms/step - loss: 0.3867 - accuracy: 0.8659 - val_loss: 0.3235 - val_accuracy: 0.8824
Epoch 4/5
344/344 [==============================] - 104s 303ms/step - loss: 0.2962 - accuracy: 0.8968 - val_loss: 0.3068 - val_accuracy: 0.9007
Epoch 5/5
344/344 [==============================] - 104s 304ms/step - loss: 0.2106 - accuracy: 0.9291 - val_loss: 0.4630 - val_accuracy: 0.8713


In [13]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
history = model.fit(train_set, steps_per_epoch=int(0.75 * dataset_size / batch_size), validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size), epochs=35, callbacks=[tensorboard_cb, early_stopping_cb])

Epoch 1/35
344/344 [==============================] - 104s 302ms/step - loss: 0.1682 - accuracy: 0.9459 - val_loss: 0.3718 - val_accuracy: 0.9173
Epoch 2/35
344/344 [==============================] - 104s 304ms/step - loss: 0.1252 - accuracy: 0.9575 - val_loss: 0.3166 - val_accuracy: 0.9044
Epoch 3/35
344/344 [==============================] - 105s 305ms/step - loss: 0.1077 - accuracy: 0.9637 - val_loss: 0.4484 - val_accuracy: 0.8952
Epoch 4/35
344/344 [==============================] - 105s 305ms/step - loss: 0.0964 - accuracy: 0.9637 - val_loss: 0.3321 - val_accuracy: 0.9154
Epoch 5/35
344/344 [==============================] - 105s 307ms/step - loss: 0.0791 - accuracy: 0.9720 - val_loss: 0.4220 - val_accuracy: 0.9191
Epoch 6/35
344/344 [==============================] - 105s 306ms/step - loss: 0.0683 - accuracy: 0.9738 - val_loss: 0.3441 - val_accuracy: 0.9136
Epoch 7/35
344/344 [==============================] - 106s 307ms/step - loss: 0.0593 - accuracy: 0.9797 - val_loss: 0.3575 -

In [14]:
model.evaluate(test_set)

46/46 [==============================] - 4s 75ms/step - loss: 0.2736 - accuracy: 0.9346


[0.2735663950443268, 0.9346048831939697]

In [15]:
tf.config.experimental.get_memory_info('GPU:0')

{'current': 177926400, 'peak': 1883904768}